# <center> 3D-Deconvnet<center>










### Configuracion de grafica e Importacion de librerias

In [1]:
# Se importan las bibliotecas
from __future__ import absolute_import, division, print_function, unicode_literals
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# lA ID de la GPU a usar, puede ser desde 0 hasta las N GPU's. Si es -1 significa que es en la CPU
os.environ["CUDA_VISIBLE_DEVICES"]="1";

import gc
import cv2
import h5py
import random

from shutil import copy2 as copy
from shutil import rmtree as delete
from IPython.display import clear_output
import numpy                    as     np
import matplotlib.pyplot as plt
import tensorflow               as     tf
from   tensorflow               import keras
from sklearn.preprocessing import MinMaxScaler
!wget -q https://raw.githubusercontent.com/JefeLitman/VideoDataGenerator/master/DatasetsLoaderUtils.py -O DatasetsLoaderUtils.py
from DatasetsLoaderUtils import flow_from_tablePaths

##### Si uso GPU

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.debugging.set_log_device_placement(False)

### Entrenando la red

##### Creando los datos del paciente para el video data generator

*Red Neuronal (Conv3D)*

In [3]:
def CNN3D(inputShape, dropout, nClass, weigh_decay):
    init = tf.keras.initializers.GlorotUniform(seed=8128)
    entrada = keras.Input(shape=inputShape, name="input_video")
    #Conv1
    #x = keras.layers.BatchNormalization(name="batch_norm_1")(entrada)
    x = keras.layers.Conv3D(filters=32, kernel_size=3, padding="valid", activation="relu", 
                          kernel_regularizer=keras.regularizers.l2(weigh_decay),
                          kernel_initializer=init,
                          name='conv3d_1')(entrada)
    x = keras.layers.MaxPool3D(pool_size=(1,2,2),strides=(1,2,2), name='max_pooling3d_1')(x)

    #Conv2
    x = keras.layers.Conv3D(filters=64, kernel_size=3, padding="valid", activation="relu", 
                          kernel_regularizer=keras.regularizers.l2(weigh_decay),
                          kernel_initializer=init,
                          name='conv3d_2')(x)
    x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(2,2,2), name='max_pooling3d_2')(x)

    #Conv3
    x = keras.layers.Conv3D(filters=128, kernel_size=3, padding="valid", activation="relu", 
                          kernel_regularizer=keras.regularizers.l2(weigh_decay),
                          kernel_initializer=init,
                          name='conv3d_3')(x)
    x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(2,2,2),name='max_pooling3d_3')(x)

    #Conv4
    x = keras.layers.Conv3D(filters=128, kernel_size=3, padding="valid", activation="relu", 
                          kernel_regularizer=keras.regularizers.l2(weigh_decay),
                          kernel_initializer=init,
                          name='conv3d_4')(x)
    x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(2,2,2),name='max_pooling3d_4')(x)

    #Conv5
    x = keras.layers.Conv3D(filters=128, kernel_size=3, padding="valid", activation="relu", 
                          kernel_regularizer=keras.regularizers.l2(weigh_decay),
                          kernel_initializer=init,
                          name='conv3d_5')(x)
    x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(2,2,2),name='max_pooling3d_5')(x)

    #fc6s
    x = keras.layers.Flatten(name='flatten_6')(x)
    x = keras.layers.Dense(2048, activation="relu", kernel_regularizer=keras.regularizers.l2(weigh_decay),
                         kernel_initializer=init,
                         name='dense_6')(x)
    x = keras.layers.Dropout(rate=dropout,name='dropout_6')(x)

    #fc7
    x = keras.layers.Dense(2048, activation="relu", kernel_regularizer=keras.regularizers.l2(weigh_decay),
                         kernel_initializer=init,
                         name='dense_7')(x)
    x = keras.layers.Dropout(rate=dropout,name='dropout_7')(x)

    #fc8
    salidas = keras.layers.Dense(nClass, activation="softmax", 
                              kernel_regularizer=keras.regularizers.l2(weigh_decay), 
                              kernel_initializer=init,
                              name='dense_8')(x)

    return keras.Model(entrada, salidas, name="LTC")

In [4]:
def cambio_lr(epoch, lr):
    if epoch == 4 or epoch == 8 :
        for i in ['conv3d_1','conv3d_2','conv3d_3','conv3d_4','conv3d_5', 'dense_6', 'dense_7', 'dense_8']:
            weigh_decay = model.get_layer(i).kernel_regularizer.get_config()['l2'] * 0.1
            model.get_layer(i).kernel_regularizer = keras.regularizers.l2(weigh_decay)
        return sgd.learning_rate.numpy() * 0.1
    else:
        return sgd.learning_rate.numpy()

In [5]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Creando los datos para cada paciente
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
for i, j in [('CG','01'),('PG','01'),('CG','02'),('PG','02'),('CG','03'),('PG','03'),
             ('CG','04'),('PG','04'),('CG','05'),('PG','05'),('CG','06'),('PG','06'),
             ('CG','07'),('PG','07'),('CG','08'),('PG','08'),('CG','09'),('PG','09'),
             ('CG','10'),('PG','10'),('CG','11'),('PG','11')]:
    
    random.seed(8128)
    np.random.seed(8128)
    tf.random.set_seed(8128)

    save_path = '/home/jefelitman/LTC_Parkinson/Models/test/'+i+'_'+j
    if not os.path.isdir(save_path):
        os.mkdir(save_path)
    
    ori_path = '/home/jefelitman/LTC_Parkinson/data2020'
    table_paths = []
    for video in sorted(os.listdir(ori_path)):
        video_path = os.path.join(ori_path,video)
        if video.split("_")[0] == i and video.split("_")[1] == j:
            table_paths.append([video_path, 'test', video.split("_")[0]])
        else:
            table_paths.append([video_path, 'train', video.split("_")[0]])

    def time_sampling(video):
        mitad = len(video)//2
        return video[mitad-30:mitad+30]
    
    def scale(video, label):
        x_min = tf.reduce_min(video, name="Min_video")
        x_max = tf.reduce_max(video, name="Max_video")
        new_video = tf.divide((video - x_min), (x_max - x_min), name="Scaling_video")
        return new_video, label
    
    dataset = flow_from_tablePaths(table_paths, time_sampling, [256, 256])
    len_train = len(dataset.__videos_train_path__)
    len_test = len(dataset.__videos_test_path__)
    
    train_data = tf.data.Dataset.from_generator(dataset.data_generator, 
                                                (tf.float32, tf.int64), 
                                                ([60, 256, 256, 1], []), 
                                                args=[1, 1])
    train_data = train_data.cache().shuffle(len_train, 
                                            reshuffle_each_iteration=True).map(scale, 24).batch(2).prefetch(1)
    
    test_data = tf.data.Dataset.from_generator(dataset.data_generator, 
                                                (tf.float32, tf.int64), 
                                                ([60, 256, 256, 1], []), 
                                                args=[2, 1])
    test_data = test_data.cache().shuffle(len_test, 
                                          reshuffle_each_iteration=True).map(scale, 24).batch(2).prefetch(1)

    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    # Entrenamiento de la red neuronal
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    name_weights = "final_model_patient_" + i + '_' + j + "_weights.h5"
    clear_output()
    print("Listo! Modelo que se va salvar: ", name_weights)
    model = CNN3D((60, 256, 256, 1), 0.5, 2, 5e-3)
    #Parametros de la red neuronal
    sgd = keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9)
    perdida = keras.losses.SparseCategoricalCrossentropy(name='loss')
    precision = keras.metrics.SparseCategoricalAccuracy(name='acc')
    model.compile(optimizer=sgd, loss = perdida, metrics = [precision])
    model_cinematic = model.fit(x = train_data,
                  epochs=10,
                  callbacks = [keras.callbacks.LearningRateScheduler(cambio_lr, verbose=1),
                               keras.callbacks.CSVLogger(os.path.join(save_path,i+'_'+j+'_output.csv'))],
                  validation_data = test_data)
    model.save(os.path.join(save_path,name_weights))

    #Grafico la perdida
    fig = plt.figure()
    plt.plot(model_cinematic.history["loss"],'k--')
    plt.plot(model_cinematic.history["val_loss"],'b--')
    plt.title('Loss over epochs')
    plt.legend(labels=["Loss","Test Loss"])
    fig.savefig(os.path.join(save_path,'train_loss.png'))
    plt.close(fig)
    #Grafico el accuracy
    fig = plt.figure()
    plt.plot(model_cinematic.history["acc"],'k--')
    plt.plot(model_cinematic.history["val_acc"],'b--')
    plt.title('Accuracy over epochs')
    plt.legend(labels=["Accuracy","Test Accuracy"])
    fig.savefig(os.path.join(save_path,'train_accuracy.png'))
    plt.close(fig)

    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    # Libero la memoria de la grafica
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    del model
    del dataset
    del train_data
    del test_data
    del table_paths
    del fig
    del model_cinematic
    del sgd
    del perdida
    del precision
    keras.backend.clear_session()
    gc.collect()

Listo! Modelo que se va salvar:  final_model_patient_CG_01_weights.h5

Epoch 00001: LearningRateScheduler reducing learning rate to 0.001.
Epoch 1/10
84/84 [==============================] - 139s 2s/step - loss: 26.8902 - acc: 0.5476 - val_loss: 26.7184 - val_acc: 0.0000e+00

Epoch 00002: LearningRateScheduler reducing learning rate to 0.001.
Epoch 2/10
84/84 [==============================] - 47s 554ms/step - loss: 26.4634 - acc: 0.5357 - val_loss: 26.3243 - val_acc: 0.0000e+00

Epoch 00003: LearningRateScheduler reducing learning rate to 0.001.
Epoch 3/10
65/84 [======================>.......] - ETA: 10s - loss: 26.0800 - acc: 0.5234

KeyError: 'val_acc'